### Crispr Amplicon

Read alignment
Consensus 
Visualisation
Stats

In [1]:
import sys
import pysam
import edlib

In [2]:
bam = "/mnt/storage1/projects/research/22070I_1057_Cas9-ONT/Sample_22070LRa005/22070LRa005.bam"
bed = "/mnt/storage1/projects/research/22070I_1057_Cas9-ONT/opsin_region.bed"
ref = "/mnt/storage2/megSAP/data/genomes/GRCh38.fa"

Read target bedfile

In [3]:
n_window = 100
roi = []
with open(bed)as f:
    for line in f:
        l = line.strip().split()
        l[1] = max(0, int(l[1]) - n_window)
        l[2] = int(l[2]) + n_window
        roi.append(l)
f.close()

roi

[['chrX', 154143540, 154295780]]

Read reference genome

In [4]:
f = pysam.FastaFile(ref)
seq_ref = f.fetch(reference = roi[0][0], start = roi[0][1], end = roi[0][2])
print("Ref Length:", len(seq_ref))

Ref Length: 152240


Read Bamfile

In [10]:
b = pysam.AlignmentFile(bam, "rb")
print("Found", b.count(contig = roi[0][0], start = roi[0][1], end = roi[0][2]), "mapped reads")

# Extract from documentation:
# Note that region strings are 1-based, while start and end denote an interval in python coordinates. The region is specified by reference, start and end.
# Todo: Maybe reduce coordinates by 1?
reads = {}
for read in b.fetch(contig = roi[0][0], start = roi[0][1], end = roi[0][2]):
    #print(read.query_name)
    reads[str(read.query_name)] = {
        'pos'   : read.get_reference_positions(full_length=True),
        'seq_query' : read.get_forward_sequence(),
        'pairs' : read.get_aligned_pairs(),
    }
b.close()

Found 288 mapped reads


Test edlib: Align single read to reference

In [12]:

testread =  next(iter(reads.values()))['seq_query']
print(testread)

None
